In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.datasets import load_iris

# Load the Iris dataset
# iris = load_iris()
# df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])

# # For simplicity, let's focus on 'sepal length (cm)' feature
# data = df['sepal length (cm)'].values
data = pd.read_csv('d:/downloadsD/Clustering_gmm.csv')
data = data['Weight'].values

# Define the negative log-likelihood function
def negative_log_likelihood(params, data):
    mu, sigma = params[0], np.abs(params[1])  # Ensure sigma is positive
    return -np.sum(norm.logpdf(data, loc=mu, scale=sigma))

# Perform Maximum Likelihood Estimation (MLE) on the original dataset
initial_guess = [np.mean(data), np.std(data)]
mle_result = minimize(negative_log_likelihood, initial_guess, args=(data,))
mle_params = mle_result.x
print(f"MLE estimates on original data: Mean = {mle_params[0]:.4f}, Std Dev = {mle_params[1]:.4f}")

# Bootstrapping process
n_bootstraps = 1000
boot_means = []
boot_stds = []

for _ in range(n_bootstraps):
    # Sample with replacement
    bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
    # Perform MLE on the bootstrap sample
    result = minimize(negative_log_likelihood, initial_guess, args=(bootstrap_sample,))
    params = result.x
    boot_means.append(params[0])
    boot_stds.append(np.abs(params[1]))

# Compute statistics from bootstrapped MLEs
mean_estimate = np.mean(boot_means)
std_dev_estimate = np.mean(boot_stds)
mean_conf_interval = np.percentile(boot_means, [2.5, 97.5])
std_conf_interval = np.percentile(boot_stds, [2.5, 97.5])

print(f"Bootstrap Mean Estimate = {mean_estimate:.4f}")
print(f"Bootstrap Std Dev Estimate = {std_dev_estimate:.4f}")
print(f"95% Confidence Interval for Mean = {mean_conf_interval}")
print(f"95% Confidence Interval for Std Dev = {std_conf_interval}")

# Plot the bootstrap distributions
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(boot_means, bins=30, color='blue', alpha=0.7)
plt.axvline(mean_estimate, color='red', linestyle='dashed', linewidth=2)
plt.title("Bootstrapped Means")
plt.xlabel("Mean")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(boot_stds, bins=30, color='green', alpha=0.7)
plt.axvline(std_dev_estimate, color='red', linestyle='dashed', linewidth=2)
plt.title("Bootstrapped Std Deviations")
plt.xlabel("Std Dev")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

KeyError: 'x'

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Scenario: Drug effect outcomes, 1 for positive effect, 0 for negative effect
data = np.array([1]*5 + [0]*3)  # 5 positive outcomes, 3 negative outcomes

# MLE for binomial distribution (estimating probability of success)
def negative_log_likelihood(p, data):
    likelihood = np.sum(data * np.log(p) + (1 - data) * np.log(1 - p))
    return -likelihood  # negative because we minimize the NLL

# Find the MLE (probability of success)
from scipy.optimize import minimize

# Initial guess for probability p
initial_p = 0.5

# Optimize to find the MLE estimate for p
result = minimize(negative_log_likelihood, initial_p, args=(data,), bounds=[(0, 1)])
p_mle = result.x[0]

print(f"MLE estimate of probability (p): {p_mle:.4f}")

# Bootstrap resampling to estimate confidence intervals
n_iterations = 1000
bootstrap_estimates = []

for i in range(n_iterations):
    bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
    result = minimize(negative_log_likelihood, initial_p, args=(bootstrap_sample,), bounds=[(0, 1)])
    bootstrap_estimates.append(result.x[0])

bootstrap_estimates = np.array(bootstrap_estimates)
confidence_interval = np.percentile(bootstrap_estimates, [2.5, 97.5])

print(f"95% Confidence Interval for probability: {confidence_interval}")

# Plot the distribution of bootstrap estimates
plt.hist(bootstrap_estimates, bins=30, edgecolor='k')
plt.axvline(p_mle, color='red', label=f'MLE Estimate (p): {p_mle:.4f}')
plt.axvline(confidence_interval[0], color='green', linestyle='--', label=f'2.5% CI: {confidence_interval[0]:.4f}')
plt.axvline(confidence_interval[1], color='green', linestyle='--', label=f'97.5% CI: {confidence_interval[1]:.4f}')
plt.title('Bootstrap Distribution of MLE Estimates')
plt.xlabel('Probability of Success (p)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Step 1: Define the dataset
data = np.array([1]*5 + [0]*3)  # 1 for success (5 people), 0 for failure (3 people)

# Step 2: Define the negative log-likelihood function for the binomial distribution
def negative_log_likelihood(p, data):
    likelihoods = data * np.log(p) + (1 - data) * np.log(1 - p)
    return -np.sum(likelihoods)

# Step 3: Bootstrap resampling
def bootstrap_resample(data, n_iterations=1000):
    n = len(data)
    estimates = []
    for i in range(n_iterations):
        sample = np.random.choice(data, size=n, replace=True)
        # Step 4: Minimize negative log-likelihood for each bootstrap sample
        result = minimize(negative_log_likelihood, x0=0.5, args=(sample,), bounds=[(0, 1)])
        estimates.append(result.x[0])  # Store the estimate of p (success probability)
    return estimates

# Step 5: Perform bootstrap MLE and calculate the confidence interval
bootstrap_estimates = bootstrap_resample(data)
mean_estimate = np.mean(bootstrap_estimates)
confidence_interval = np.percentile(bootstrap_estimates, [2.5, 97.5])

# Output results
print(f"Mean Estimate of Success Probability: {mean_estimate:.3f}")
print(f"95% Confidence Interval for Success Probability: {confidence_interval}")

# Visualize the bootstrap distribution of estimates
plt.hist(bootstrap_estimates, bins=30, edgecolor='black')
plt.title('Bootstrap Distribution of Success Probability Estimates')
plt.xlabel('Success Probability')
plt.ylabel('Frequency')
plt.show()
